In [1]:
import pandas as pd
import torch
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = '/content/drive/My Drive/Colab Notebooks/Housing.csv'
sample = pd.DataFrame(pd.read_csv(path))
print(sample.shape)
sample.head(3)

(545, 13)


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished


In [3]:
# Converting the strings to values

list1 =  ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']
list2 = ['furnishingstatus']

# Defining the map function
def map1(x):
    return x.map({'yes': 1, 'no': 0})

def map2(x):
    return x.map({'furnished': 1, 'semi-furnished': 0})

sample[list1] = sample[list1].apply(map1)
sample[list2] = sample[list2].apply(map2)

sample.head(3)

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,1.0
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,1.0
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,0.0


In [4]:
X = sample.iloc[:, 1:12].values
Y = sample.iloc[:, 0].values
X[:1]

array([[7420,    4,    2,    3,    1,    0,    0,    0,    1,    2,    1]])

In [5]:
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
X = scale.fit_transform(X)
X[:1]

array([[ 1.04672629,  1.40341936,  1.42181174,  1.37821692,  0.40562287,
        -0.46531479, -0.73453933, -0.2192645 ,  1.4726183 ,  1.51769249,
         1.80494113]])

In [18]:
from sklearn.model_selection import train_test_split
import numpy as np

# Splitting the training and validation sets.
x_train, x_test, y_train, y_test = train_test_split(X, Y, train_size = 0.8, test_size = 0.2, random_state = 0)

In [19]:
ten_train_x = torch.tensor(x_train)
ten_train_y = torch.tensor(y_train)
ten_test_x = torch.tensor(x_test)
ten_test_y = torch.tensor(y_test)

In [20]:
w = torch.ones((12))
w

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [25]:
def model(ten_train_x, w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, b):
  # Use matrix multiplication for all data points in ten_train_x
  return ten_train_x @ torch.tensor([w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10], dtype=torch.double) + b # @ represents matrix multiplication
  #return w * ten_train_x + b

def lossf(pred, ten_train_y):
  squared_diff = (pred-ten_train_y)**2
  return squared_diff.mean()

In [26]:
def training_loop(n_epochs, optimizer, prm, x_train, x_test, y_train, y_test):
  for epoch in range(1, n_epochs + 1):

    train_pred = model(x_train, *prm)
    train_loss = lossf(train_pred, y_train)

    with torch.no_grad():
      test_pred = model(x_test, *prm)
      test_loss = lossf(test_pred, y_test)
      assert test_loss.requires_grad == False

    optimizer.zero_grad()
    train_loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
      print(f"Epoch {epoch}, Training Loss {train_loss.item():.4f}", f'Validation loss {test_loss.item():.4f}')
  return prm

In [28]:
prm = torch.tensor([1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0], requires_grad=True)
learning_rate = 1e-4
optimizer = torch.optim.SGD([prm], lr=learning_rate)


training_loop(
              n_epochs=5000,
              optimizer = optimizer,
              prm = prm,
              x_train = ten_train_x,
              x_test = ten_test_x,
              y_train = ten_train_y,
              y_test = ten_test_y
              )

TypeError: model() takes 13 positional arguments but 14 were given